# Priporočilni sistem

In [1]:
from csv import DictReader
from datetime import datetime
import numpy as np
from operator import itemgetter, attrgetter
import itertools

%matplotlib inline
%config InlineBackend.figure_formats = ['jpg']
import matplotlib
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.style.use('PR.mplstyle')

In [2]:
videogames_data = {}


with open('../../podatki/video_game.csv', 'rt', encoding='utf-8') as data:
    for row in DictReader(data):
        videogames_data[row['Rank']] = row
                        

# Primer branja iz slovarja podatkov
for key, value in videogames_data.items():
    for atribut, vrednost in value.items():
        print(atribut + ": " + vrednost)
    break

Rank: 1
Name: Wii Sports
Platform: Wii
Year_of_Release: 2006
Genre: Sports
Publisher: Nintendo
NA_players: 41.36
EU_players: 28.96
JP_players: 3.77
Other_players: 8.45
Global_players: 82.53
Critic_Score: 76
Critic_Count: 51
User_Score: 8
User_Count: 322
Developer: Nintendo
Rating: E


#### Filtriranje podatkov glede na izbrane kategorije (Platform, minimal Year_of_Release, Genre, Publisher, Developer, Rating)

Tukaj nastavimo parametre za filtriranje podatkov. Če katerega od parametrov ne želimo opredeliti, parameter nastavimo na prezen niz, kar pomeni, da se pri filtriranju ne bo upošteval.

In [3]:
platform = "PS4"
min_year = "2012"
max_year = "2015"
genre = "Sports"
publisher = "Electronic Arts"
developer = "EA Sports"
rating = "E"

In [4]:
filtrirane_igre = {}

for key, value in videogames_data.items():
    is_platform  = False
    is_min_year  = False
    is_max_year  = False
    is_genre     = False
    is_publisher = False
    is_developer = False
    is_rating    = False

    for atribut, vrednost in value.items():
        if (atribut == "Platform" and platform != "" and vrednost == platform):
            is_platform = True
        else:
            if platform == "":
                is_platform = True
           
        if (atribut == "Year_of_Release" and min_year != "" and vrednost >= min_year):
            is_min_year = True
        else:
            if min_year == "":
                is_min_year = True
                
        if (atribut == "Year_of_Release" and max_year != "" and vrednost <= max_year):
            is_max_year = True
        else:
            if max_year == "":
                is_max_year = True
            
        if (atribut == "Genre" and genre != "" and vrednost == genre):
            is_genre = True
        else:
            if genre == "":
                is_genre = True
        
        if (atribut == "Publisher" and publisher != "" and vrednost == publisher):
            is_publisher = True
        else:
            if publisher == "":
                is_publisher = True
        
        if (atribut == "Developer" and developer != "" and vrednost == developer):
            is_developer = True
        else:
            if developer == "":
                is_developer = True
            
        if (atribut == "Rating" and rating != "" and vrednost == rating):
            is_rating = True
        else:
            if rating == "":
                is_rating = True
                
    if is_platform and is_min_year and is_max_year and is_genre and is_publisher and is_developer and is_rating:   
        filtrirane_igre[key] = value
        
        
for key, value in filtrirane_igre.items():
    for atribut, vrednost in value.items():
        print(f'{atribut}: {vrednost}, ', end="")
    print()
    print()

Rank: 78, Name: FIFA 16, Platform: PS4, Year_of_Release: 2015, Genre: Sports, Publisher: Electronic Arts, NA_players: 1.12, EU_players: 6.12, JP_players: 0.06, Other_players: 1.28, Global_players: 8.57, Critic_Score: 82, Critic_Count: 42, User_Score: 4.3, User_Count: 896, Developer: EA Sports, Rating: E, 

Rank: 144, Name: FIFA 15, Platform: PS4, Year_of_Release: 2014, Genre: Sports, Publisher: Electronic Arts, NA_players: 0.8, EU_players: 4.33, JP_players: 0.05, Other_players: 0.9, Global_players: 6.08, Critic_Score: 82, Critic_Count: 47, User_Score: 5.7, User_Count: 988, Developer: EA Sports, Rating: E, 

Rank: 421, Name: Madden NFL 16, Platform: PS4, Year_of_Release: 2015, Genre: Sports, Publisher: Electronic Arts, NA_players: 2.34, EU_players: 0.3, JP_players: 0, Other_players: 0.6, Global_players: 3.24, Critic_Score: 83, Critic_Count: 37, User_Score: 5.9, User_Count: 180, Developer: EA Sports, Rating: E, 

Rank: 750, Name: Madden NFL 15, Platform: PS4, Year_of_Release: 2014, Genre

### Priporočanje filtriranih iger, glede na Global_sales, Critic_Score, User_Score

Filtrirane podatke, nato tukaj obdelam ter izračunam razmerja Global_sales, Critic_Score, User_Score in na podlagi teh rezultatov sistem priporoči in izpiše najboljše igre v padajočem vrstnem reku za ta filter podatkov.

In [5]:
najbolj_primerni = []

for key, value in filtrirane_igre.items():
    sales = filtrirane_igre[key]['Global_players']
    critic_score = filtrirane_igre[key]['Critic_Score']
    critic_count = filtrirane_igre[key]['Critic_Count']
    user_score = filtrirane_igre[key]['User_Score']
    user_count = filtrirane_igre[key]['User_Count']
    name = filtrirane_igre[key]['Name']
    
#     print(f'{sales} {critic_score} {critic_count} {user_score} {user_count}')
    if sales == "" or sales == "NA" or \
       critic_score == "" or critic_score == "NA" or \
       critic_count == "" or critic_count == "NA" or \
       user_score == "" or user_score == "NA" or \
       user_count == "" or user_count == "NA":
        continue
    
    sales = float(filtrirane_igre[key]['Global_players'])
    critic_score = float(filtrirane_igre[key]['Critic_Score'])
    critic_count = float(filtrirane_igre[key]['Critic_Count'])
    user_score = float(filtrirane_igre[key]['User_Score'])
    user_count = float(filtrirane_igre[key]['User_Count'])
    
    ratio_critic = (critic_score / critic_count) * 0.35
    ratio_user = (user_score / user_count) * 0.5 # user_count / user_score
    ratio_sales = sales * 0.15 * (max(ratio_critic, ratio_user) / min(ratio_critic, ratio_user))
    
#     print(f'{name} {ratio_critic} {ratio_user} {ratio_sales}')
#     print()
    najbolj_primerni.append([value, ratio_sales])
    
najbolj_primerni = sorted(najbolj_primerni, reverse=True, key=itemgetter(1))

for key, value in najbolj_primerni:
    name = key['Name']
    print(f'{name}: {value}')
    print()

FIFA 16: 366.078511627907

FIFA 15: 193.05940425531912

Madden NFL 16: 23.2825836005497

Madden NFL 15: 17.16525299069561

Madden NFL 25: 12.206715976331362

NBA Live 14: 2.8093333333333326

NBA Live 15: 0.9117343750000002

NBA Live 16: 0.47393442622950815

